In [1]:
import numpy as np
import time

from collections import namedtuple

In [2]:
'''
struct to storage order price and volume to trade

Usage:
a = Order(price=1, volume=1)
'''

Order = namedtuple('order', 'id price volume')

In [3]:
class Dynamic_order:
    '''
    extension of Order, used in making deals
    '''
    def __init__(self, order):
        assert isinstance(order, Order)
        self.id = order.id
        self.price = order.price
        self.volume = order.volume
    @property
    def show(self):
        print('id : %d' % self.id)
        print('price : %d' % self.price)
        print('volume : %d' % self.volume)

In [4]:
class Trader:
    '''
    A primary class to define an investor, including actions
    like gennerate an order, and update it's own status with
    the trading result
    '''
    def __init__(self, id):
        self.id = id
        self.cash = 0 # 现金
        self.cash_history = []
        self.asset = 0 # 总资产
        self.asset_history = []
        self.stock = 0 # 持股
        self.stock_history = []
        self.trade_history = [] # 交易记录
        self.price = 0
        self.order = Order(id=self.id,
                           price=0,
                           volume=0)
        
    def gen_order(self):
        self.order = Order(id=self.id,
                           price=np.random.randint(-10, 10), 
                           volume=np.random.randint(1, 5))
        
    def update(self, price, vol):
        vol *= np.sign(self.order.price)
        price = abs(price)
        self.cash -= price * vol
        self.cash_history.append(self.cash)
        self.stock += vol
        self.stock_history.append(self.stock)
        self.trade_history.append((price, vol))
        self.price = price
        self.asset = self.cash + self.stock * self.price
        self.asset_history.append(self.asset)
        
    @property
    def show(self):
        print('id : %d, cash : %d, stock : %d, stock_price : %d, asset : %d' % 
             (self.id, self.cash, self.stock, self.price, self.asset))
        
#         print('asset : %d' % self.asset)
#         print('stock : %d' % self.stock)
        

In [12]:
class Market:
    '''
    A class to define a market stucture, initialize with a 
    number of investors, have the ability to generate orders
    and deal with the orders from traders, returns a deal_price 
    and volume for every trader and update their status
    
    The history of price and volume is also contained, as well
    as indices like MA and so on
    '''
    def __init__(self, num_investors):
        self.investors = [Trader(i) for i in range(num_investors)]
        self.price = []
        self.vol = []
        self.MA_5 = []
        self.MA_10 = []
        self.MA_100 = []
        self.MA_500 = []
        
    def make_deals(self, orders):
        buy_list = [Dynamic_order(order) for order in orders if order.price>0]
        sell_list = [Dynamic_order(order) for order in orders if order.price<0]

        buy_list = sorted(buy_list, key=lambda x:(-x.price, x.id))
        sell_list = sorted(sell_list, key=lambda x:(abs(x.price), x.id))
        deal_result = np.zeros(len(orders), int)
        price = 0
        if len(buy_list)==0 or len(sell_list)==0:
            return price, deal_result
        while sell_list[0].price+buy_list[0].price>=0:
            if sell_list[0].volume>=buy_list[0].volume:
                sell_list[0].volume -= buy_list[0].volume
                deal_result[buy_list[0].id] += buy_list[0].volume
                deal_result[sell_list[0].id] += buy_list[0].volume
                if sell_list[0].volume==buy_list[0].volume:
                    price = (buy_list[0].price + abs(sell_list[0].price)) / 2
                else:
                    price = abs(sell_list[0].price)
                _ = buy_list.pop(0)
            else:
                buy_list[0].volume -= sell_list[0].volume
                deal_result[buy_list[0].id] += sell_list[0].volume
                deal_result[sell_list[0].id] += sell_list[0].volume
                price = buy_list[0].price
                _ = sell_list.pop(0)           
            price = abs(price)
            if len(buy_list)==0 or len(sell_list)==0:
                break
        self.update(price, deal_result)
        return price, deal_result 
    
    def update(self, price, deal_result):
        self.price.append(price)
        self.vol.append(np.sum(deal_result))
        duration = len(self.price)
        self.MA_5.append(np.mean(self.price[-5:]) if duration>5 else 0)
        self.MA_10.append(np.mean(self.price[-10:]) if duration>10 else 0)
        self.MA_100.append(np.mean(self.price[-100:]) if duration>100 else 0)
        self.MA_500.append(np.mean(self.price[-500:]) if duration>500 else 0)
        
    def step(self):
        orders = []
        for investor in self.investors:
            investor.gen_order()
            orders.append(investor.order)

        price, deal_result = self.make_deals(orders)
        for i,investor in enumerate(self.investors):
            investor.update(price=price, vol=deal_result[i])
                    

In [13]:
market = Market(30)

In [14]:
for i in range(30):
    market.step()

In [18]:
for investor in market.investors:
    investor.show

id : 0, cash : 89, stock : -12, stock_price : 6, asset : 17
id : 1, cash : -85, stock : 12, stock_price : 6, asset : -13
id : 2, cash : -64, stock : 9, stock_price : 6, asset : -10
id : 3, cash : -20, stock : 4, stock_price : 6, asset : 4
id : 4, cash : 26, stock : -1, stock_price : 6, asset : 20
id : 5, cash : -48, stock : 10, stock_price : 6, asset : 12
id : 6, cash : -24, stock : 3, stock_price : 6, asset : -6
id : 7, cash : 11, stock : -1, stock_price : 6, asset : 5
id : 8, cash : 7, stock : -2, stock_price : 6, asset : -5
id : 9, cash : 14, stock : -7, stock_price : 6, asset : -28
id : 10, cash : 15, stock : -2, stock_price : 6, asset : 3
id : 11, cash : -46, stock : 14, stock_price : 6, asset : 38
id : 12, cash : -30, stock : 1, stock_price : 6, asset : -24
id : 13, cash : -69, stock : 11, stock_price : 6, asset : -3
id : 14, cash : 60, stock : -16, stock_price : 6, asset : -36
id : 15, cash : -29, stock : 6, stock_price : 6, asset : 7
id : 16, cash : -81, stock : 11, stock_price

In [19]:
market.MA_10

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6.0999999999999996,
 6.0,
 6.5,
 6.2999999999999998,
 5.7999999999999998,
 5.7000000000000002,
 5.9000000000000004,
 5.9000000000000004,
 5.7999999999999998,
 5.5999999999999996,
 5.7000000000000002,
 5.5999999999999996,
 5.2999999999999998,
 5.5,
 5.9000000000000004,
 5.7999999999999998,
 5.9000000000000004,
 5.4000000000000004,
 5.5,
 5.5]